In [ ]:
# !pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = "https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrList.do?pgmid=HIRAA030069000400"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# "심의사례공개" 탭은 가장 마지막 테이블일 가능성 높음
tables = soup.select("div.tb-type01")
review_case_table = tables[-1]

rows = review_case_table.select("tbody tr")
data = []

for row in rows:
    cols = row.find_all("td")
    title_tag = cols[2].find("a")
    title = title_tag.text.strip()
    onclick = title_tag['onclick']
    file_icon = cols[3].find("i")
    file_type = file_icon['title'] if file_icon else ''
    date = cols[4].text.strip()
    views = cols[5].text.strip()

    m = re.search(r"viewInsuAdtCrtr\((\d+),\s*'(\d+)',\s*'(\d+)',\s*'(\d+)',\s*'(\d+)'\)", onclick)
    if m:
        seq, date_code, gubun, docid, listid = m.groups()
        data.append({
            "title": title,
            "date": date,
            "views": views,
            "file_type": file_type,
            "date_code": date_code,
            "gubun": gubun,
            "docid": docid,
            "listid": listid,
        })

df = pd.DataFrame(data)
df.head()


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrList.do?pgmid=HIRAA030069000400"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

def collect_all_items(tabGbn=10, decIteTpCd=10, max_page=118):
    results = []
    for page in range(1, max_page + 1):
        data = {
            "pageIndex": str(page),
            "tabGbn": str(tabGbn),
            "decIteTpCd": str(decIteTpCd),
            "searchYn": "Y",
            "recordCountPerPage": "10"
        }
        res = requests.post(url, headers=headers, data=data)
        soup = BeautifulSoup(res.text, "html.parser")
        rows = soup.select("div.tb-type01 > table > tbody > tr")

        for row in rows:
            title_el = row.select_one("td.col-tit a")
            results.append({
                "제목": title_el.text.strip() if title_el else "",
                "작성일": row.select_one("td.col-date").text.strip(),
                "조회수": row.select_one("td.col-views").text.strip(),
                "분류": row.select_one("td.col-gubun").text.strip()
            })
        print(f"✅ {page}페이지 수집 완료")
    return results

all_items = collect_all_items()
df = pd.DataFrame(all_items)
df.to_csv("심의사례공개_전체목록.csv", index=False, encoding="utf-8-sig")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrList.do?pgmid=HIRAA030069000400"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

def collect_all_items(tabGbn=10, decIteTpCd=10, max_page=118):
    results = []
    for page in range(1, max_page + 1):
        data = {
            "pageIndex": str(page),
            "tabGbn": str(tabGbn),
            "decIteTpCd": str(decIteTpCd),
            "searchYn": "Y",
            "recordCountPerPage": "10"
        }
        res = requests.post(url, headers=headers, data=data)
        soup = BeautifulSoup(res.text, "html.parser")
        rows = soup.select("div.tb-type01 > table > tbody > tr")

        for row in rows:
            title_el = row.select_one("td.col-tit a")
            results.append({
                "제목": title_el.text.strip() if title_el else "",
                "작성일": row.select_one("td.col-date").text.strip(),
                "조회수": row.select_one("td.col-views").text.strip(),
                "분류": row.select_one("td.col-gubun").text.strip()
            })
        print(f"✅ {page}페이지 수집 완료")
    return results

all_items = collect_all_items()
df = pd.DataFrame(all_items)
df.to_csv("심의사례공개_전체목록.csv", index=False, encoding="utf-8-sig")


In [ ]:
import os
import re
import requests
from bs4 import BeautifulSoup

BASE_DIR = "C:\Jimin\cg_SURI-Agent\data_pipeline\data"
SAVE_FILE_NM = "심의사례_첨부파일"
SAVE_DIR = os.path.join(BASE_DIR, SAVE_FILE_NM)
os.makedirs(SAVE_DIR, exist_ok=True)

base_url = "https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrList.do?pgmid=HIRAA030069000400"
detail_base = "https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrDtl.do"

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

def parse_onclick_args(onclick_text):
    args = re.findall(r"'(.*?)'", onclick_text)
    if len(args) == 4:
        rn = re.search(r'\d+', onclick_text.split(',')[0]).group()
        return {
            "pgmid": "HIRAA030069000400",
            "tabGbn": "10",
            "mtgHmeDd": args[0],
            "mtgMtrRegSno": args[1],
            "sno": args[2],
            "RN": rn
        }
    return None

def collect_and_download(max_page=118):
    for page in range(1, max_page + 1):
        data = {
            "pageIndex": str(page),
            "tabGbn": "10",
            "decIteTpCd": "10",
            "searchYn": "Y",
            "recordCountPerPage": "10"
        }
        res = requests.post(base_url, headers=headers, data=data)
        soup = BeautifulSoup(res.text, "html.parser")
        rows = soup.select("div.tb-type01 > table > tbody > tr")

        for row in rows:
            a_tag = row.select_one("td.col-tit a")
            if not a_tag: continue
            onclick = a_tag.get("onclick", "")
            meta = parse_onclick_args(onclick)
            if not meta: continue

            title = a_tag.text.strip().replace("/", "_").replace("?", "")
            detail_res = requests.get(detail_base, params=meta)
            detail_soup = BeautifulSoup(detail_res.text, "html.parser")

            file_links = detail_soup.select("div.tb-type01 td.col-file a")
            for idx, file_tag in enumerate(file_links):
                file_url = file_tag.get("href")
                if not file_url.startswith("http"):
                    file_url = "https://www.hira.or.kr" + file_url

                ext = os.path.splitext(file_url)[1]
                filename = f"{title}_{idx+1}{ext}"
                filepath = os.path.join(SAVE_DIR, filename)

                try:
                    with requests.get(file_url, stream=True) as f:
                        with open(filepath, 'wb') as out:
                            out.write(f.content)
                    print(f"📥 저장 완료: {filename}")
                except Exception as e:
                    print(f"❌ 다운로드 실패: {filename} → {e}")
        print(f"✅ {page}페이지 처리 완료\n")

collect_and_download()


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm

url = "https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrList.do?pgmid=HIRAA030069000400"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "User-Agent": "Mozilla/5.0"
}

def collect_all_items(tabGbn=10, decIteTpCd=10, max_page=118):
    results = []
    for page in tqdm(range(1, max_page + 1), desc="📄 목록 페이지"):
        data = {
            "pageIndex": str(page),
            "tabGbn": str(tabGbn),
            "decIteTpCd": str(decIteTpCd),
            "searchYn": "Y",
            "recordCountPerPage": "10"
        }
        res = requests.post(url, headers=headers, data=data)
        soup = BeautifulSoup(res.text, "html.parser")
        rows = soup.select("div.tb-type01 > table > tbody > tr")

        if not rows:
            break

        for row in rows:
            title_el = row.select_one("td.col-tit a")
            title = title_el.text.strip() if title_el else ""
            onclick = title_el.get("onclick", "") if title_el else ""

            # onclick 파라미터 파싱
            match = re.search(r"viewInsuAdtCrtr\(\s*['\"]?(\d+)['\"]?,\s*['\"]?(\d{8})['\"]?,\s*['\"]?(\d+)['\"]?,\s*['\"]?(\d{4})['\"]?,\s*['\"]?(\d{2})['\"]?\)", onclick)
            if match:
                docid, mtgHmeDd, sno, mtgMtrRegSno, _ = match.groups()
                popup_url = f"https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrPopup.do?mtgHmeDd={mtgHmeDd}&sno={sno}&mtgMtrRegSno={mtgMtrRegSno}"
            else:
                docid = mtgHmeDd = sno = mtgMtrRegSno = popup_url = ""

            results.append({
                "제목": title,
                "작성일": row.select_one("td.col-date").text.strip(),
                "조회수": row.select_one("td.col-views").text.strip(),
                "분류": row.select_one("td.col-gubun").text.strip(),
                "docid": docid,
                "mtgHmeDd": mtgHmeDd,
                "sno": sno,
                "mtgMtrRegSno": mtgMtrRegSno,
                "popup_url": popup_url
            })
    return results

# 실행
all_items = collect_all_items()
df = pd.DataFrame(all_items)
df.to_csv("/content/심의사례공개_전체목록.csv", index=False, encoding="utf-8-sig")
print(f"✅ 총 {len(df)}건 저장 완료 → /content/심의사례공개_전체목록.csv")


📄 목록 페이지: 100%|██████████| 118/118 [09:24<00:00,  4.78s/it]


OSError: Cannot save file into a non-existent directory: '\content'

In [ ]:
df.to_csv("./심의사례공개_전체목록.csv", index=False, encoding="utf-8-sig")
print(f"✅ 총 {len(df)}건 저장 완료 → /심의사례공개_전체목록.csv")

✅ 총 1172건 저장 완료 → /content/심의사례공개_전체목록.csv


In [9]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm

# === 설정 ===
INPUT_CSV = "./심의사례공개_전체목록.csv"
SAVE_DIR = "./data"
BASE_URL = "https://www.hira.or.kr"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# 디렉토리 준비
os.makedirs(SAVE_DIR, exist_ok=True)

# 데이터 로드
df = pd.read_csv(INPUT_CSV)

# 첨부파일 수집 함수
def download_file(popup_url, filename_base):
    try:
        res = requests.get(popup_url, headers=HEADERS)
        soup = BeautifulSoup(res.text, "html.parser")
        attach_link = soup.select_one("a.btn_file")

        if not attach_link:
            return None

        onclick = attach_link.get("onclick", "")
        import re
        match = re.search(r"goDown1\('([^']+)','([^']+)'\)", onclick)
        if not match:
            return None

        rel_url, original_name = match.groups()
        download_url = urljoin(BASE_URL, rel_url)
        ext = os.path.splitext(original_name)[-1]
        save_path = os.path.join(SAVE_DIR, f"{filename_base}{ext}")

        # 파일 다운로드
        r = requests.get(download_url, headers=HEADERS)
        with open(save_path, "wb") as f:
            f.write(r.content)
        return save_path

    except Exception as e:
        print(f"❌ 오류: {e}")
        return None

# 전체 실행
for _, row in tqdm(df.iterrows(), total=len(df), desc="📎 첨부파일 다운로드"):
    popup_url = row["popup_url"]
    base_name = f"{row['mtgHmeDd']}_{row['sno']}_{row['mtgMtrRegSno']}"
    path = download_file(popup_url, base_name)
    if path:
        print(f"✅ 저장됨: {path}")
    else:
        print(f"⚠️ 첨부파일 없음: {popup_url}")


📎 첨부파일 다운로드:   0%|          | 0/1172 [00:00<?, ?it/s]

❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan


📎 첨부파일 다운로드:   1%|          | 10/1172 [00:01<02:15,  8.55it/s]

✅ 저장됨: ./data\20250530.0_1.0_4.0.pdf


📎 첨부파일 다운로드:   1%|          | 11/1172 [00:01<03:47,  5.11it/s]

...

⚠️ 첨부파일 없음: https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrPopup.do?mtgHmeDd=20241031&sno=1&mtgMtrRegSno=0015


📎 첨부파일 다운로드:  78%|███████▊  | 914/1172 [01:00<00:00, 539.73it/s]

⚠️ 첨부파일 없음: https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrPopup.do?mtgHmeDd=20241031&sno=1&mtgMtrRegSno=0016
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps yo

📎 첨부파일 다운로드: 100%|██████████| 1172/1172 [01:00<00:00, 19.42it/s]

❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 오류: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨

In [11]:
import os
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote
from tqdm import tqdm

# === 설정 ===
CSV_FILE = "./심의사례공개_전체목록.csv"
SAVE_DIR = "첨부파일"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# 저장 디렉토리 생성
os.makedirs(SAVE_DIR, exist_ok=True)

# 팝업에서 첨부파일 다운로드 URL 추출
def get_real_download_urls(popup_url):
    try:
        res = requests.get(popup_url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        urls = []

        for a in soup.select('a.btn_file[onclick*="Header.goDown1"]'):
            onclick = a.get("onclick", "")
            m = re.search(r"Header\.goDown1\('([^']+)'\s*,\s*'([^']+)'\)", onclick)
            if not m:
                continue
            rel_src, fnm = m.groups()
            fnm_enc = quote(fnm, safe='')
            download_url = (
                "https://www.hira.or.kr/download.do?"
                + urlencode({"src": rel_src, "fnm": fnm_enc})
            )
            urls.append((download_url, fnm))
        return urls
    except Exception as e:
        print(f"❌ 팝업 접근 실패: {popup_url} / {e}")
        return []

# 실제 다운로드 수행
def download_file(url, save_name):
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        content_type = r.headers.get("Content-Type", "")
        if "text/html" in content_type:
            print(f"⚠️ HTML 오류 페이지 감지됨 (무시): {save_name}")
            return False
        with open(os.path.join(SAVE_DIR, save_name), "wb") as f:
            f.write(r.content)
        return True
    except Exception as e:
        print(f"❌ 다운로드 실패: {url} / {e}")
        return False

# 전체 실행 플로우
def main():
    df = pd.read_csv(CSV_FILE)

    for _, row in tqdm(df.iterrows(), total=len(df), desc="📎 첨부파일 수집"):
        popup_url = row["popup_url"]
        prefix = f"{row['mtgHmeDd']}_{row['sno']}_{row['mtgMtrRegSno']}"

        urls = get_real_download_urls(popup_url)
        if not urls:
            print(f"⚠️ 첨부파일 없음: {popup_url}")
            continue

        for i, (url, original_name) in enumerate(urls):
            ext = os.path.splitext(original_name)[-1]
            # 여러 파일이 있을 수 있으니 번호 추가
            suffix = f"_{i+1}" if len(urls) > 1 else ""
            save_name = f"{prefix}{suffix}{ext}"
            success = download_file(url, save_name)
            if success:
                print(f"✅ 저장 완료: {save_name}")

if __name__ == "__main__":
    main()


📎 첨부파일 수집:   0%|          | 0/1172 [00:00<?, ?it/s]

❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan


📎 첨부파일 수집:   1%|          | 10/1172 [00:00<01:10, 16.43it/s]

✅ 저장 완료: 20250530.0_1.0_4.0.pdf
✅ 저장 완료: 20250530.0_1.0_5.0.pdf


📎 첨부파일 수집:   1%|          | 12/1172 [00:01<03:53,  4.97it/s]

...

⚠️ 첨부파일 없음: https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrPopup.do?mtgHmeDd=20241031&sno=1&mtgMtrRegSno=0015


📎 첨부파일 수집:  72%|███████▏  | 842/1172 [01:02<00:00, 794.76it/s]

⚠️ 첨부파일 없음: https://www.hira.or.kr/rc/insu/insuadtcrtr/InsuAdtCrtrPopup.do?mtgHmeDd=20241031&sno=1&mtgMtrRegSno=0016
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supp

📎 첨부파일 수집: 100%|██████████| 1172/1172 [01:02<00:00, 18.68it/s]  

❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
⚠️ 첨부파일 없음: nan
❌ 팝업 접근 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps

In [12]:
import os
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote
from tqdm import tqdm

# === 설정 ===
CSV_FILE = "./심의사례공개_전체목록.csv"
SAVE_DIR = "첨부파일2"
JSON_OUTPUT = "심의사례_구조화.json"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# 저장 폴더 생성
os.makedirs(SAVE_DIR, exist_ok=True)

# 파일명 생성 함수
def get_safe_filename(published_date, category, title, index, ext):
    # 불가 문자 제거
    safe_title = re.sub(r'[\\/:*?"<>|]', '', title)
    safe_title = safe_title.replace(" ", "").strip()[:30]
    return f"{published_date}_{category}_{safe_title}_{index+1}{ext}"

# 팝업 상세 정보 추출
def parse_popup(row):
    popup_url = row["popup_url"]
    try:
        res = requests.get(popup_url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        # 본문 내용 추출
        content_el = soup.select_one("div.popup_cont, div.detail_cont, div.cont-area, div#content")
        content = content_el.get_text(strip=True, separator="\n") if content_el else ""

        # 첨부파일 추출
        file_urls = []
        saved_files = []
        for i, a in enumerate(soup.select('a.btn_file[onclick*="Header.goDown1"]')):
            onclick = a.get("onclick", "")
            m = re.search(r"Header\.goDown1\('([^']+)'\s*,\s*'([^']+)'\)", onclick)
            if not m:
                continue
            rel_src, fnm = m.groups()
            fnm_enc = quote(fnm, safe='')
            download_url = (
                "https://www.hira.or.kr/download.do?"
                + urlencode({"src": rel_src, "fnm": fnm_enc})
            )
            file_urls.append(download_url)

            # 실제 다운로드 시도
            ext = os.path.splitext(fnm)[-1]
            save_name = get_safe_filename(
                row["작성일"], row["분류"], row["제목"], i, ext
            )
            save_path = os.path.join(SAVE_DIR, save_name)

            try:
                r = requests.get(download_url, headers=HEADERS, timeout=10)
                if "text/html" in r.headers.get("Content-Type", ""):
                    print(f"⚠️ HTML 오류 감지 (스킵): {save_name}")
                    continue
                with open(save_path, "wb") as f:
                    f.write(r.content)
                saved_files.append(save_name)
                print(f"✅ 저장 완료: {save_name}")
            except Exception as e:
                print(f"❌ 다운로드 실패: {download_url} / {e}")

        # 관련근거 추정
        relevant = ""
        for line in content.splitlines():
            if "근거" in line or "관련" in line:
                relevant = line.strip()
                break

        return {
            "published_date": row["작성일"],
            "title": row["제목"],
            "content": content,
            "file_urls": file_urls,
            "saved_files": saved_files,
            "category": row["분류"],
            "relevant": relevant
        }

    except Exception as e:
        print(f"❌ 팝업 파싱 실패: {popup_url} / {e}")
        return None

# 전체 실행
def main():
    df = pd.read_csv(CSV_FILE)
    records = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 구조화 및 다운로드"):
        result = parse_popup(row)
        if result:
            records.append(result)

    with open(JSON_OUTPUT, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)
    print(f"\n✅ 전체 완료: {len(records)}건 구조화 → {JSON_OUTPUT}")

if __name__ == "__main__":
    main()


📄 구조화 및 다운로드:   0%|          | 0/1172 [00:00<?, ?it/s]

❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?


📄 구조화 및 다운로드:   1%|          | 10/1172 [00:01<02:06,  9.22it/s]

✅ 저장 완료: 2025-05-30_공개사례_면역관용요법(ImmuneToleranceInductio_1.pdf


📄 구조화 및 다운로드:   1%|          | 11/1172 [00:11<26:53,  1.39s/it]

...

✅ 저장 완료: 2024-10-31_공개사례_심율동전환제세동기거치술(ICD)및심장재동기화치료(CRT_1.pdf


📄 구조화 및 다운로드:  45%|████▌     | 530/1172 [01:19<00:01, 410.99it/s]

❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'n

📄 구조화 및 다운로드: 100%|██████████| 1172/1172 [01:19<00:00, 14.75it/s]

❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
❌ 팝업 파싱 실패: nan / Invalid URL 'n

In [ ]:
import os
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote
from tqdm import tqdm

# === 설정 ===
CSV_FILE = "심의사례공개_전체목록.csv"
SAVE_DIR = "첨부파일3"
JSON_OUTPUT = "심의사례_구조화.json"
FAILURE_LOG = "failed_popups.csv"
HEADERS = {"User-Agent": "Mozilla/5.0"}

os.makedirs(SAVE_DIR, exist_ok=True)

def get_safe_filename(published_date, category, title, index, ext):
    safe_title = re.sub(r'[\\/:*?"<>|]', '', title)
    safe_title = safe_title.replace(" ", "").strip()[:30]
    return f"{published_date}_{category}_{safe_title}_{index+1}{ext}"

def parse_popup(row):
    popup_url = row["popup_url"]
    try:
        res = requests.get(popup_url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        content_el = soup.select_one("div.popup_cont, div.detail_cont, div.cont-area, div#content")
        content = content_el.get_text(strip=True, separator="\n") if content_el else ""

        file_urls = []
        saved_files = []
        for i, a in enumerate(soup.select('a.btn_file[onclick*="Header.goDown1"]')):
            onclick = a.get("onclick", "")
            m = re.search(r"Header\.goDown1\('([^']+)'\s*,\s*'([^']+)'\)", onclick)
            if not m:
                continue
            rel_src, fnm = m.groups()
            fnm_enc = quote(fnm, safe='')
            download_url = (
                "https://www.hira.or.kr/download.do?"
                + urlencode({"src": rel_src, "fnm": fnm_enc})
            )
            file_urls.append(download_url)

            ext = os.path.splitext(fnm)[-1]
            save_name = get_safe_filename(row["작성일"], row["분류"], row["제목"], i, ext)
            save_path = os.path.join(SAVE_DIR, save_name)

            try:
                r = requests.get(download_url, headers=HEADERS, timeout=10)
                if "text/html" in r.headers.get("Content-Type", ""):
                    print(f"⚠️ HTML 오류 감지 (스킵): {save_name}")
                    continue
                with open(save_path, "wb") as f:
                    f.write(r.content)
                saved_files.append(save_name)
                print(f"✅ 저장 완료: {save_name}")
            except Exception as e:
                print(f"❌ 다운로드 실패: {download_url} / {e}")

        relevant = ""
        for line in content.splitlines():
            if "근거" in line or "관련" in line:
                relevant = line.strip()
                break

        return {
            "published_date": row["작성일"],
            "title": row["제목"],
            "content": content,
            "file_urls": file_urls,
            "saved_files": saved_files,
            "category": row["분류"],
            "relevant": relevant
        }

    except Exception as e:
        print(f"❌ 팝업 파싱 실패: {popup_url} / {e}")
        return None

def main():
    df = pd.read_csv(CSV_FILE)
    records = []
    failures = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="📄 구조화 및 다운로드"):
        result = parse_popup(row)
        if result:
            records.append(result)
        else:
            failures.append({
                "제목": row["제목"],
                "작성일": row["작성일"],
                "분류": row["분류"],
                "popup_url": row["popup_url"]
            })

    with open(JSON_OUTPUT, "w", encoding="utf-8") as f:
        json.dump(records, f, ensure_ascii=False, indent=2)

    if failures:
        pd.DataFrame(failures).to_csv(FAILURE_LOG, index=False, encoding="utf-8-sig")
        print(f"⚠️ 팝업 파싱 실패 {len(failures)}건 → {FAILURE_LOG} 저장 완료")

    print(f"\n✅ 전체 성공 건수: {len(records)} → {JSON_OUTPUT}")

if __name__ == "__main__":
    main()
